In [25]:
import os
import glob
import shutil
import pandas as pd

import pymdwizard
from pymdwizard.core import xml_utils
from pymdwizard.core.xml_utils import xml_node
from pymdwizard.core import spatial_utils

In [35]:
## Metadata record to be edited
md_fname = r"Z:\FORT_DataManagement\Shafroth_2017_ElwhaEstuary\Composition_MetadataCreate_FGDC.xml"
## A crosswalk table to get from the attribute label to the full scientific name that will be appended 
## to the Attribute Description element
fname = r"Z:\FORT_DataManagement\Shafroth_2017_ElwhaEstuary\Elwha Estuary_species list for data release.xlsx"

In [36]:
## Read in the crosswalk table to a pandas dataframe
lookup_df = pd.read_excel(fname, "Sheet1")
lookup_df

,Code,Scientific name
0,ACERCIR,Acer circinatum
1,ACERMAC,Acer macrophyllum
2,ACHIMIL,Achillea millefolium
3,ADENBIC,Adenocaulon bicolor
4,AGROCAP,Agrostis capillaris
5,AGROEXA,Agrostis exarata ssp. minor
6,AGROSTO,Agrostis stolonifera
7,AIRACAR,Aira caryophyllea
8,ALISPLA,Alisma plantago-aquatica
9,ALNURUB,Alnus rubra


In [37]:
## A lookup function that extracts the scientific name from the crosswalk table
code = 'ACERCIR'
def lookup(code):
    return lookup_df.loc[lookup_df['Code'] == code, 'Scientific name'].iloc[0]
lookup(code)


'Acer circinatum'

In [38]:
## The root node of our metadata record 
md = xml_utils.fname_to_node(md_fname)
## A list of the 'attr' nodes in our metadata record
attr_list = md.xpath('eainfo/detailed/attr')

In [39]:
## The text string that is constant throughout the applicable attribute definitions
attrdef_text = "Percent living cover in plot of"

In [40]:
rdom = attr.xpath('attrdomv/rdom')
rdom

[<Element rdom at 0x8940f88>]

In [41]:
## Loop through all applicable attributes (starting in position 3 in this case), replacing the values in Attribute Definition
## with the text from 'attrdef_text' concatenated with the scientific names looked up from the crosswalk table.
## Add the Attribute Units of Measure element for each attribute and populate with 'Percent'.
for attr in attr_list[3:]:
    attrlabl_list = attr.xpath('attrlabl')
    attrlabl_node = attr.xpath('attrlabl')[0]
    attrlabl = attrlabl_node.text
    print (attrlabl)
    sciname = lookup(attrlabl)
    
    attrdef = attr.xpath('attrdef')[0]
    new_attrdef = attrdef_text  + " " + sciname
    attrdef.text = new_attrdef
    
    rdom = attr.xpath('attrdomv/rdom')[0]
    attrunit = xml_node('attrunit', 'Percent', rdom)

ACERCIR
ACERMAC
ACHIMIL
ADENBIC
AGROCAP
AGROEXA
AGROSTO
AIRACAR
ALISPLA
ALNURUB
AMBRCHA
ANAPMAR
ANTEDIM
ARBUMEN
ARTESUK
ARUNDIO
ASPLTRI
ATHYFIL
BROMDIA
BROMINE
BROMSIT
BROMVUL
CAKIMAR
CARDOLG
CAREDEW
CARELEN
CARELYN
CAREOBN
CARESTI
CHAMANG
CIRCALP
CIRSARV
CIRSVUL
CLAYSIB
CRATDOU
CRATMON
CYTISCO
DACTGLO
DESCELO
DIGIPUR
ELEOPAL
ELYMREP
EPILCIL
EPIPGIG
EQUIARV
EQUIFLU
EQUIHYE
FESTRUB
FESTSUB
FESTVIR
FRAGVIR
GALIAPA
GALIBOR
GALIKAM
GALITRI
GAULSHA
GERAROB
HEDEHEL
HIERALB
HOLCLAN
HOLODIS
HYPEPER
HYPORAD
IMPAESC
IRISPSE
JUNCACU
JUNCARC
JUNCART
JUNCBAL
JUNCBOL
JUNCBUF
JUNCEFF
JUNCENS
LAPSCOM
LATHJAP
LATHPAL
LATHSYL
LEPICAM
LEPIVIR
LEUCVUL
LEYMMOL
LONICIL
LONIINV
LOTUCOR
MAIADIL
MALUFUS
MATRDIS
MENTARV
MENTXPIP
MIMULEW
MYCEMUR
MYOSSCO
MYRIAQU
OEMLCER
OENASAR
OSMOBER
PETAFRI
PHALARU
PHLEPRA
PIPEELO
PLANLAN
PLANMAJ
PLANMAR
POA_PAL
POA_TRV
POLYGLY
POLYMUN
POPUTRI
POTEANS
PRUNVUL
PSEUMEN
PTERAQU
RANUREP
RIBEDIV
RIBELAC
RORIISL
ROSANUT
ROSAPIS
RUBUARM
RUBUPAR
RUBUSPE
RUBUURS
RUMEACE
RUMECRI
RUMESAL

In [42]:
## Save out an XML document
xml_utils.save_to_file(md, r"Z:\FORT_DataManagement\Shafroth_2017_ElwhaEstuary\Composition_MetadataCreate_TEST.xml")